# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://robby.iex.cloud/v1/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2321237697660,
 'week52high': 178.3,
 'week52low': 123.98,
 'week52highSplitAdjustOnly': 179.61,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': -0.10351797852500566,
 'sharesOutstanding': 15821946000,
 'float': 0,
 'avg10Volume': 54577302,
 'avg30Volume': 70612566,
 'day200MovingAvg': 146.8,
 'day50MovingAvg': 144.78,
 'employees': 164000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.9170023129162208,
 'dividendYield': 0.006250441775722314,
 'nextDividendDate': '',
 'exDividendDate': '2023-02-10',
 'nextEarningsDate': '2023-05-04',
 'peRatio': 24.39017870632861,
 'beta': 1.3052200029310859,
 'maxChangePercent': 55.96148470259357,
 'year5ChangePercent': 2.511976291586673,
 'year2ChangePercent': 0.228724910008526,
 'year1ChangePercent': -0.10351797852500566,
 'ytdChangePercent': 0.1308706648095035,
 'month6ChangePercent': -0.13330891561709868,
 'month3ChangePercent': -0.006687980457393383,
 'month1ChangePercent': 0.03576791625748288,
 'day30Cha

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://robby.iex.cloud/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'       
        ],
        index = my_columns),
            ignore_index = True
        )

In [7]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAPL,146.71,-0.103518,N/A
1,MSFT,249.22,-0.153527,N/A
2,AMZN,93.5,-0.392022,N/A
3,BRK.B,304.02,-0.047676,N/A
4,GOOGL,89.13,-0.337124,N/A
...,...,...,...,...
498,LNC,31.86,-0.520298,N/A
499,DVA,82.31,-0.281763,N/A
500,FOX,33.06,-0.122865,N/A
501,DISH,13.27,-0.595427,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)

## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \nPlease try again:')
        portfolio_size =  input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)    

Enter the size of your portfolio:1000000
1000000


In [15]:
position_size = float(portfolio_size)/len(final_dataframe.index)
    
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,347,FSLR,162.16,1.294284,123
1,264,CEG,79.27,0.822236,252
2,337,STLD,118.5,0.74695,168
3,131,MPC,124.92,0.672054,160
4,139,VLO,132.75,0.613582,150
5,427,EQT,34.73,0.549146,575
6,154,OXY,58.98,0.535186,339
7,398,LW,99.98,0.492134,200
8,313,PWR,160.64,0.487823,124
9,19,MRK,109.89,0.486143,182


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [16]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile', 
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://robby.iex.cloud/v1/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A', 
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
hqm_dataframe 

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,146.71,N/A,-0.103518,N/A,-0.133309,N/A,-0.006688,N/A,0.035768,N/A,N/A
1,MSFT,249.22,N/A,-0.153527,N/A,-0.101494,N/A,0.009512,N/A,0.038378,N/A,N/A
2,AMZN,93.5,N/A,-0.392022,N/A,-0.31891,N/A,0.000963,N/A,-0.037868,N/A,N/A
3,BRK.B,304.02,N/A,-0.047676,N/A,0.019756,N/A,-0.042246,N/A,-0.025265,N/A,N/A
4,GOOGL,89.13,N/A,-0.337124,N/A,-0.235919,N/A,-0.085471,N/A,-0.063957,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,LNC,31.86,N/A,-0.520298,N/A,-0.341579,N/A,-0.168192,N/A,-0.028659,N/A,N/A
499,DVA,82.31,N/A,-0.281763,N/A,-0.106879,N/A,0.113501,N/A,0.062339,N/A,N/A
500,FOX,33.06,N/A,-0.122865,N/A,0.031977,N/A,0.122581,N/A,0.055219,N/A,N/A
501,DISH,13.27,N/A,-0.595427,N/A,-0.285022,N/A,-0.170625,N/A,-0.096049,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [17]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
    

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,146.71,N/A,-0.103518,0.361829,-0.133309,0.192843,-0.006688,0.514911,0.035768,0.823062,N/A
1,MSFT,249.22,N/A,-0.153527,0.284294,-0.101494,0.276342,0.009512,0.590457,0.038378,0.836978,N/A
2,AMZN,93.5,N/A,-0.392022,0.043738,-0.31891,0.019881,0.000963,0.540755,-0.037868,0.345924,N/A
3,BRK.B,304.02,N/A,-0.047676,0.461233,0.019756,0.592445,-0.042246,0.347913,-0.025265,0.44334,N/A
4,GOOGL,89.13,N/A,-0.337124,0.06163,-0.235919,0.067594,-0.085471,0.17495,-0.063957,0.218688,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,LNC,31.86,N/A,-0.520298,0.017893,-0.341579,0.013917,-0.168192,0.037773,-0.028659,0.421471,N/A
499,DVA,82.31,N/A,-0.281763,0.115308,-0.106879,0.264414,0.113501,0.89662,0.062339,0.908549,N/A
500,FOX,33.06,N/A,-0.122865,0.335984,0.031977,0.61829,0.122581,0.906561,0.055219,0.878728,N/A
501,DISH,13.27,N/A,-0.595427,0.00994,-0.285022,0.035785,-0.170625,0.033797,-0.096049,0.081511,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [18]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,146.71,N/A,-0.103518,0.361829,-0.133309,0.192843,-0.006688,0.514911,0.035768,0.823062,0.473161
1,MSFT,249.22,N/A,-0.153527,0.284294,-0.101494,0.276342,0.009512,0.590457,0.038378,0.836978,0.497018
2,AMZN,93.5,N/A,-0.392022,0.043738,-0.31891,0.019881,0.000963,0.540755,-0.037868,0.345924,0.237575
3,BRK.B,304.02,N/A,-0.047676,0.461233,0.019756,0.592445,-0.042246,0.347913,-0.025265,0.44334,0.461233
4,GOOGL,89.13,N/A,-0.337124,0.06163,-0.235919,0.067594,-0.085471,0.17495,-0.063957,0.218688,0.130716
...,...,...,...,...,...,...,...,...,...,...,...,...
498,LNC,31.86,N/A,-0.520298,0.017893,-0.341579,0.013917,-0.168192,0.037773,-0.028659,0.421471,0.122763
499,DVA,82.31,N/A,-0.281763,0.115308,-0.106879,0.264414,0.113501,0.89662,0.062339,0.908549,0.546223
500,FOX,33.06,N/A,-0.122865,0.335984,0.031977,0.61829,0.122581,0.906561,0.055219,0.878728,0.684891
501,DISH,13.27,N/A,-0.595427,0.00994,-0.285022,0.035785,-0.170625,0.033797,-0.096049,0.081511,0.040258


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [19]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop=True)

## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the size of your portfolio:
That is not a number! 
Please try again:
Enter the size of your portfolio:1000000


In [21]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,URI,450.43,44,0.428672,0.970179,0.414098,0.992048,0.258678,0.978131,0.151405,0.984095,0.981113
1,ACGL,69.17,289,0.454985,0.974155,0.462677,0.996024,0.177763,0.954274,0.067438,0.920477,0.961233
2,RE,386.17,51,0.306912,0.950298,0.402111,0.99006,0.154746,0.934394,0.100701,0.968191,0.960736
3,STLD,118.5,168,0.74695,0.996024,0.376946,0.986083,0.143534,0.930417,0.073272,0.930417,0.960736
4,WYNN,105.53,189,0.202347,0.894632,0.624788,1.0,0.408381,0.992048,0.070393,0.924453,0.952783
5,GEHC,75.01,266,0.24004,0.922465,0.24004,0.94831,0.24004,0.976143,0.09169,0.960239,0.951789
6,BWA,49.78,401,0.195945,0.882704,0.2913,0.964215,0.183303,0.95825,0.109922,0.970179,0.943837
7,PH,348.03,57,0.19436,0.87674,0.211165,0.930417,0.141555,0.928429,0.126209,0.978131,0.928429
8,GWW,664.1,30,0.414268,0.968191,0.148109,0.868787,0.101915,0.864811,0.189305,0.988072,0.922465
9,ULTA,519.99,38,0.37538,0.960239,0.240286,0.950298,0.159786,0.944334,0.034538,0.817097,0.917992


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)


## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [24]:
column_formats ={
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template], 
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [25]:
writer.save()